In [1]:
import time
import numpy as np
import datajoint as dj
import networkx as nx

# Visualizing the offline neuron

In [3]:
import ipyvolume as ipv
import numpy as np
import trimesh
%matplotlib inline
ipv.figure()
# we draw the tetrahedron
neuron_data = np.load("107816118160698192_neuron_2_visualizations.npz")
original_mesh_verts = neuron_data["original_mesh_verts"]
original_mesh_faces = neuron_data["original_mesh_faces"]
edges = neuron_data["edges"]
unique_skeleton_verts = neuron_data["unique_skeleton_verts"]


original_mesh = trimesh.Trimesh(vertices=original_mesh_verts,faces=original_mesh_faces)


mesh = ipv.plot_trisurf(original_mesh.vertices[:,0], 
                        original_mesh.vertices[:,1],
                        original_mesh.vertices[:,2], 
                        triangles=original_mesh.faces, color='orange')

mesh.color = [0., 1., 0., 0.5]
mesh.material.transparent = True
# and also mark the vertices
mesh2 = ipv.plot_trisurf(unique_skeleton_verts[:,0], 
                        unique_skeleton_verts[:,1], 
                        unique_skeleton_verts[:,2], 
                        lines=edges, color='blue')

volume_maxs = np.max(original_mesh.vertices,axis=0)
volume_mins = np.min(original_mesh.vertices,axis=0)
ranges = volume_maxs - volume_mins
index = [0,1,2]
max_index = np.argmax(ranges)
min_limits = [0,0,0]
max_limits = [0,0,0]

buffer = 10000
for i in index:
    if i == max_index:
        min_limits[i] = volume_mins[i] - buffer
        max_limits[i] = volume_maxs[i] + buffer 
        continue
    else:
        difference = ranges[max_index] - ranges[i]
        min_limits[i] = volume_mins[i] - difference/2  - buffer
        max_limits[i] = volume_maxs[i] + difference/2 + buffer

#ipv.xyzlim(-2, 2)
ipv.xlim(min_limits[0],max_limits[0])
ipv.ylim(min_limits[1],max_limits[1])
ipv.zlim(min_limits[2],max_limits[2])
ipv.show()

In [31]:
# Example of how to put a skeleton into a networkx mdoel
print(unique_skeleton_verts)
edges

[[1220760.  655666.  891025.]
 [1222000.  656940.  890723.]
 [1222000.  656940.  890723.]
 ...
 [1350390.  623455.  928625.]
 [1350390.  623455.  928625.]
 [1350390.  623481.  928566.]]


array([[   0,    1],
       [   2,    3],
       [   4,    5],
       ...,
       [7904, 7905],
       [7906, 7907],
       [7908, 7909]])

In [27]:
B = nx.Graph()
B.add_nodes_from([(x,{"coordinates":y}) for x,y in enumerate(unique_skeleton_verts)])
B.add_edges_from(edges_with_coefficients)

#now calculate the weights of the edge from 

In [37]:
# how to create it from adjacency matrix
# G=nx.from_numpy_matrix(A)

In [38]:
G = nx.Graph()
G.add_edges_from(original_mesh.face_adjacency)

In [40]:
original_mesh.face_adjacency

array([[355536, 355845],
       [106653, 173406],
       [160384, 275959],
       ...,
       [294484, 295031],
       [294482, 294483],
       [294483, 294484]])

In [62]:
edges_as_vertices = original_mesh.vertices[original_mesh.edges]
distances = np.linalg.norm(edges_as_vertices[:,1]  - edges_as_vertices[:,0]
                           ,axis=1)
distances.shape

(1343724,)

In [64]:
G = nx.Graph()

In [ ]:
G.add_weighted_edges_from

# Shortest Path algorithm from trimesh example:

In [70]:
current_mesh = trimesh.load_mesh("107816118160698192_top_apical_smaller.off")

In [71]:
mesh_lists = current_mesh.split(only_watertight = False)

In [72]:
import trimesh

import networkx as nx
import time 

start_time = time.time()

# test on a sphere mesh
mesh = mesh_lists[0]

# edges without duplication
edges = mesh.edges_unique

# the actual length of each unique edge
length = mesh.edges_unique_length

# create the graph with edge attributes for length
g = nx.Graph()
# for edge, L in zip(edges, length):
#     g.add_edge(*edge, length=L)

# alternative method for weighted graph creation
# you can also create the graph with from_edgelist and
# a list comprehension, which is like 1.5x faster
ga = nx.from_edgelist([(e[0], e[1], {'length': L})
                       for e, L in zip(edges, length)])

nx.diameter(ga)

print("Total time for diameter = " + str(start_time - time.time()))

# # arbitrary indices of mesh.vertices to test with
# start = 0
# end = int(len(mesh.vertices) / 2.0)

# # run the shortest path query using length for edge weight
# path = nx.shortest_path(g,
#                         weight='length')

# # VISUALIZE RESULT
# # make the sphere transparent-ish
# mesh.visual.face_colors = [100, 100, 100, 100]
# # Path3D with the path between the points
# path_visual = trimesh.load_path(mesh.vertices[path])
# # visualizable two points
# points_visual = trimesh.points.PointCloud(mesh.vertices[[start, end]])

# # create a scene with the mesh, path, and points
# scene = trimesh.Scene([
#     points_visual,
#     path_visual,
#     mesh])

# scene.show(smooth=False)

Total time for diameter = -744.4620568752289
